In [ ]:
#Inspired by https://github.com/maetshju/gsoc2018

In [24]:
using Flux: onehotbatch
using DSP

using FileIO: load, save, loadstreaming, savestreaming
import LibSndFile

using MFCC
using Plots
using BSON

┌ Info: Precompiling Flux [587475ba-b771-5e3f-ad9e-33799f191a9c]
└ @ Base loading.jl:1260


In [ ]:
#print(pwd()*"/Datasets/ravdess-emotional-speech-audio/audio_speech_actors_01-24/Actor_01/03-01-01-01-01-01-01.wav")

FRAME_LENGTH = 0.025 # ms
FRAME_INTERVAL = 0.010 # ms

In [59]:
#Step 2 
"""
    getFeatures( wavName )
Extracts Mel filterbanks and associated labels from `wavName`, fortunately ravdess incorporates labels into the filename
"""
function getFeatures( wavName )
    
    samps, sr = load(wavName)
    samps = vec(samps)
    
    #.+ eps to avoid Inf's and NaN's
    samps.+=eps()

    #Kind of handles resampling! Consider lowering quality of training data if accuracy is too off
    mfccs, _, _ = mfcc(samps, sr, :rasta; wintime=FRAME_LENGTH, steptime=FRAME_INTERVAL)
    
    #Directories might be missing on Repo
    #GET 06!!!
    #emotion = 03-01-06-01-02-01-12.wav
    emotion = parse(Float64,(wavName[82:83]))
    
    #plot(mfccs)
    #print(mfccs)
    
    #use this if you want to use deltas as additional features
    #use this if you're not going to use a Recurrent Network
    #mfccDeltas = deltas(mfccs, 2)
    #features = hcat(mfccs, mfccDeltas)
    
    #We're using a Recurrent Network
    features = mfccs
    
    #print(size(features))
    
    
    labels = fill( emotion, length(features) )
    
    toReturn = (features, labels)
    return toReturn
end

#typeof(getFeatures("Datasets/ravdess-emotional-speech-audio/audio_speech_actors_01-24/Actor_01/03-01-01-01-01-01-01.wav"))

getFeatures

In [60]:
"""
    createData(data_dir, out_dir)
Extracts data from files in `data_dir` and saves results in `out_dir`.
"""
function createData(data_dir, out_dir)
    for (root, dirs, files) in walkdir(data_dir)
        #println(files)
        
        
        wavFiles = [x for x in files if occursin("wav",x)]
        
        one_dir_up = basename(root)
        #println(wavFiles)
                    
        for wavFile in wavFiles
            wavPath = joinpath(root, wavFile)
            #println(wavPath)
            #return
            x, y = getFeatures(wavPath)
            classes = [n for n in 1:8]
            #print(classes)
            y = onehotbatch(y, classes)'
                        
            base, _ = splitext(wavFile)
            #print(one_dir_up * "-" * base * ".bson")
            dat_name = one_dir_up * "-" * base * ".bson"
            dat_path = joinpath(out_dir, dat_name)
            #println(dat_path)
            BSON.@save dat_path x y
        end
    end
end

createData

In [61]:
createData("Datasets/ravdess-emotional-speech-audio","Datasets/ravdess-processed")

In [ ]:
#createData(TRAINING_DATA_DIR, TRAINING_OUT_DIR)
#createData(TEST_DATA_DIR, TEST_OUT_DIR)